This notebook uses the CSVs created with the OULAD - 1 - Feature engineering notebook. Make sure to run the feature engineering code before continuing with this notebook. Additionally, this notebook uses the non-standard library [PyTorch](https://pytorch.org/), which you may need to install before proceeding.

We furthermore use the 'pass-fail' scenario as an example for this notebook. Results for the other OULAD scenarios can easily be obtained by altering the CSV file names.

### Import libraries and define functions to split data over local clients

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def region_train_test_split(df, X, y, test_size, seed):
    np.random.seed(seed)
    
    regions = df.columns[38:51].values
    
    X_train, X_test, y_train, y_test = None, None, None, None
    
    for idx, region in enumerate(regions):
        region_indices = df[df[region] == 1].index.values
        np.random.shuffle(region_indices)
        
        region_size = len(region_indices)
        train_index = int(region_size * (1 - test_size))
        
        train_indices = region_indices[:train_index]
        test_indices = region_indices[train_index:]
        
        if idx == 0:
            X_train = X[train_indices].copy()
            X_test = X[test_indices].copy()
            y_train = y[train_indices]
            y_test = y[test_indices]
        else:
            X_train = np.append(X_train, X[train_indices].copy(), axis = 0)
            X_test = np.append(X_test, X[test_indices].copy(), axis = 0)
            y_train = np.append(y_train, y[train_indices])
            y_test = np.append(y_test, y[test_indices])
        
    return X_train, X_test, y_train, y_test

In [ ]:
def split_into_torch_clients(X, y, n_clients, region_dummy, seed = 42):
    n_samples = len(X)
    
    assert n_samples == len(y), 'Number of samples in X and y must be the same.'
    assert n_clients > 0, 'Number of clients must be greater than 0.'
    assert n_clients <= n_samples, 'Number of clients cannot be greater than number of samples.'

    np.random.seed(seed)
    clients_indices = []
    if region_dummy:
        for col_idx in range(38,51):
            column = X[:, col_idx]
            # Due to scaling should not check for equals 1, but > 0
            clients_indices.append(np.where(column > 0)[0])
    else:
        random_indices = np.random.choice(n_samples, n_samples, replace = False)
        clients_indices = np.array_split(random_indices, n_clients)
    
    clients = []
    for client_indices in clients_indices:
        X_client = X[client_indices].copy()
        y_client = y[client_indices]
        
        X_torch = torch.tensor(X_client, dtype=torch.float32)
        y_torch = torch.tensor(y_client, dtype=torch.int64)

        clients.append((X_torch, y_torch))

    return clients

### Define federated learning functions

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(in_dim,30)
        self.hidden_layer1  = nn.Linear(30,10)
        self.output_layer   = nn.Linear(10,out_dim)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out
    
def federated_averaging(global_model, local_models, num_clients, client_sizes):
    dataset_size = sum(client_sizes)
    
    for param_global, params_local in zip(global_model.parameters(), zip(*[model.parameters() for model in local_models])):
        weighted_sum = torch.zeros_like(param_global.data)
        for client_params, client_size in zip(params_local, client_sizes):
            client_weight = client_size / dataset_size
            weighted_sum += client_weight * client_params.data
        
        param_global.data = weighted_sum
        
def train_local_model(model, dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            prediction = model(data)
            loss = loss_fn(prediction, target)
            loss.backward()
            optimizer.step()
            
def validate_local_model(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in dataloader:
            prediction = model(data)
            total_loss += loss_fn(prediction, target).item()
            _, predicted_labels = torch.max(prediction, 1)
            correct_predictions += (predicted_labels == target).sum().item()
            total_samples += len(target)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples

    return average_loss, accuracy

In [ ]:
def federated_learning(clients, input_dim, output_dim, loss_fn, lr, optim_str, num_clients, num_rounds, num_epochs, batch_size):
    # use device = torch.device("cuda" if torch.cuda.is_available() else "cpu") to enable gpu
    device = torch.device("cpu")
    global_model = NeuralNetwork(input_dim, output_dim).to(device)
    
    client_sizes = [len(client) for client in clients]
    for r in range(num_rounds):
        local_models = []
        local_optimizers = []
        local_dataloaders = []
        for i in range(num_clients):
            local_model = NeuralNetwork(input_dim, output_dim).to(device)
            local_model.load_state_dict(global_model.state_dict())
            local_optimizer = optim.SGD(local_model.parameters(), lr=lr)
            if optim_str == 'adam':
                local_optimizer = optim.Adam(local_model.parameters(), lr=lr)
            
            X, y = clients[i]
            X, y = X.to(device), y.to(device)
            local_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y),
                                                           batch_size=batch_size, shuffle=True)
            train_local_model(local_model, local_dataloader, local_optimizer, loss_fn, num_epochs, device)
            
            local_models.append(local_model)
            local_optimizers.append(local_optimizer)
            local_dataloaders.append(local_dataloader)
            
        federated_averaging(global_model, local_models, num_clients, client_sizes)
        if r % 10 == 0:
            for i in range(num_clients):
                X, y = clients[i]
                X, y = X.to(device), y.to(device)
                local_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y),
                                                               batch_size=batch_size, shuffle=False)
                val_loss, val_accuracy = validate_local_model(local_models[i], local_dataloader, loss_fn)
                print(f"Client {i+1} - Round {r + 1}/{num_rounds}, Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}")
            
    return global_model

### Read data and train model

In [ ]:
df = pd.read_csv('oulad_pass_fail_x.csv')
X = pd.read_csv('oulad_pass_fail_x.csv').to_numpy()
y = pd.read_csv('oulad_pass_fail_y.csv').to_numpy().ravel()

input_dim = X.shape[1]
output_dim = len(np.unique(y))

scaler = StandardScaler()
X = scaler.fit_transform(X)

In this code example we set the number of clients to 10. Changing N_CLIENTS to another value will easily yield the results for other local client numbers.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
lr = 0.02
optimizer = 'adam'

REGION_DUMMY = False
N_CLIENTS = 10
if REGION_DUMMY:
    N_CLIENTS = 13
    
N_ROUNDS = 50
N_EPOCHS = 2
BATCH_SIZE = 64

acc_fed, f1_fed = [], []
for i in range(10):
    X_main, X_holdout, y_main, y_holdout = None, None, None, None
    if REGION_DUMMY:
        X_main, X_holdout, y_main, y_holdout = region_train_test_split(df, X, y, test_size = 0.2, seed = i)
    else:
        X_main, X_holdout, y_main, y_holdout = train_test_split(X, y, test_size = 0.2, random_state = i)

    torch_clients = split_into_torch_clients(X_main, y_main, N_CLIENTS, REGION_DUMMY)
    global_model = federated_learning(torch_clients, input_dim, output_dim, loss_fn, lr, optimizer, N_CLIENTS, N_ROUNDS, N_EPOCHS, BATCH_SIZE)
    
    global_model.eval()
    # use device = torch.device("cuda" if torch.cuda.is_available() else "cpu") to enable gpu
    device = torch.device("cpu")
    with torch.no_grad():
        predictions = global_model(torch.tensor(X_holdout, dtype=torch.float32).to(device))

    _, y_pred = torch.max(predictions, dim=1)
    y_probs = F.softmax(predictions, dim=1)

    y_pred = y_pred.cpu().numpy()
    y_probs = y_probs.cpu().numpy()
    
    acc_fed.append(accuracy_score(y_holdout, y_pred))
    f1_fed.append(f1_score(y_holdout, y_pred))

### Store results as CSV

In [ ]:
df = pd.DataFrame({'acc': acc_fed, 'f1': f1_fed})

df.to_csv('oulad_pass_fail_federated_10clients.csv', index = False)